In [ ]:
WORKSPACE = "/workspace"
#WORKSPACE = "/root/jupyter/report_portal/workspace"
YAML = WORKSPACE + "/benchmark_config.yaml"
BASE_DATASTORE = WORKSPACE + "/base.datastore.json"
TEST_DATASTORE = WORKSPACE + "/test.datastore.json"
BASE_METADATA = WORKSPACE + "/base.testrun_metadata.json"
TEST_METADATA = WORKSPACE + "/test.testrun_metadata.json"

BASE_TESTRUN_RESULT = WORKSPACE + "/base.testrun_result.csv"
TEST_TESTRUN_RESULT = WORKSPACE + "/test.testrun_result.csv"
METADATA = WORKSPACE + "/2way_metadata.csv"
BENCHMARK = WORKSPACE + "/2way_benchmark.csv"

import os
import sys
import yaml
import json
import pandas as pd
from IPython.display import display, HTML, Markdown
from datetime import datetime

BASEPATH = os.path.abspath('.')
SCRIPTPATH = BASEPATH + "/fetch_scripts"

# round data to two decimals
FORMATER="{:.2f}".format

In [ ]:
%%HTML
<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.

In [ ]:
def printmd(string, color=None):
    colorstr ="<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))

# Performance Test Report Portal

In [ ]:
now = datetime.now()
dt_string = "Generate time: *{}*".format(now.strftime("%Y-%m-%d %H:%M:%S"))
printmd(dt_string)

## Introduction

In [ ]:
def read_json(json_file):
    with open(json_file, 'r') as f:
        try:
            data = json.load(f)
        except Exception as e:
            print("Fail to load {}".format(json_file))
            raise
    return data

base_metadata = read_json(BASE_METADATA)
test_metadata = read_json(TEST_METADATA)
assert base_metadata.get("testrun.type") == test_metadata.get("testrun.type"), "Base and Test type must be the same! Exit."

run_type = base_metadata.get("testrun.type")
base_platform = base_metadata.get("testrun.platform")
test_platform = test_metadata.get("testrun.platform")
# Type and platform must not be None
assert run_type is not None, "Type is None! Exit."
assert base_platform is not None, "Base platform is None! Exit."
assert test_platform is not None, "Test platform is None! Exit."

with open('{}/templates/{}_{}.md'.format(BASEPATH, base_platform.lower(), run_type), 'r') as f:
    display(Markdown(f.read()))
    
if base_platform != test_platform:
    with open('{}/templates/{}_{}.md'.format(BASEPATH, test_platform.lower(), run_type), 'r') as f:
        display(Markdown('\n'+f.read()))

In [ ]:
%%html
<style>
    table {
        display: inline-block
    }
</style>

In [ ]:
# Generate Base testrun result
!python3 $SCRIPTPATH/generate_testrun_results.py \
    --config $YAML \
    --datastore $BASE_DATASTORE \
    --metadata $BASE_METADATA \
    --output $BASE_TESTRUN_RESULT
assert os.path.exists(BASE_TESTRUN_RESULT), "Fail to generate {}! Exit.".format(BASE_TESTRUN_RESULT)

# Generate Test testrun result
!python3 $SCRIPTPATH/generate_testrun_results.py \
    --config $YAML \
    --datastore $TEST_DATASTORE \
    --metadata $TEST_METADATA \
    --output $TEST_TESTRUN_RESULT
assert os.path.exists(TEST_TESTRUN_RESULT), "Fail to generate {}! Exit.".format(TEST_TESTRUN_RESULT)

# Generate 2way metadata
!python3 $SCRIPTPATH/generate_2way_metadata.py \
    --test $TEST_METADATA \
    --base $BASE_METADATA \
    --output $METADATA
assert os.path.exists(METADATA), "Fail to generate {}! Exit.".format(METADATA)

#Generate 2way benchmark
!python3 $SCRIPTPATH/generate_2way_benchmark.py \
    --config $YAML \
    --test $TEST_TESTRUN_RESULT \
    --base $BASE_TESTRUN_RESULT \
    --output $BENCHMARK
assert os.path.exists(BENCHMARK), "Fail to generate {}! Exit.".format(BENCHMARK)

In [ ]:
def color_delta(val):
    color_dict = {
        "Major Regression": 'red',
        "Minor Regression": 'black',
        "Major Improvement": 'green',
        "Minor Improvement": 'black',
        "Variance Too Large": 'orange',
        "No Significance": 'black',
        
    }
    return 'color: {}'.format(color_dict.get(val, 'black'))

def highlight_cols(s):
    return 'background-color: #eeffff'

def bold_font(s):
    return 'font-weight: bold'

def displayComparison(df):
    #These are the columns which need special formatting
    deltacols=df.columns.map(lambda x: x.endswith("-SPEC"))
    roundcols=df.columns.map(lambda x: x.endswith(("-AVG", "-%SD", "-%DIFF", "-SIGN")))
    display(df.style.applymap(color_delta,subset=deltacols).applymap(bold_font,subset=deltacols).format(FORMATER, subset=roundcols).hide_index())

## Metadata

In [ ]:
%%HTML
* The differences between Test and Base are <b style='color:orange'>highlighted</b>.

In [ ]:
def highlight_diff(row, cell_format):
    cell_format = cell_format if row['TEST'] != row['BASE'] else ''
    format_row = ['', cell_format, cell_format]
    return format_row

def color_diff(row):
    return highlight_diff(row, 'color: orange')

def bold_diff(row):
    return highlight_diff(row, 'font-weight: bold')
    
conf_df = pd.read_csv(METADATA, index_col=0)
conf_df = conf_df[['KEY', 'TEST', 'BASE']]
sorter = ['testrun.id'] + [x for x in conf_df['KEY'] if x != 'testrun.id']
conf_df['KEY'] = conf_df['KEY'].astype("category")
conf_df["KEY"].cat.set_categories(sorter, inplace=True)
conf_df.sort_values(['KEY'], inplace=True)
display(conf_df.style.applymap(bold_font, subset=['KEY']).apply(color_diff, axis=1).apply(bold_diff, axis=1).hide_index())

## Summary

In [ ]:
with open('{}/templates/summary_introduction.html'.format(BASEPATH), 'r') as f:
    display(HTML(f.read()))

In [ ]:
benchmark_df = pd.read_csv(BENCHMARK, index_col=0)
summary_df = benchmark_df[['RW','BS','IOdepth','Numjobs']+list(benchmark_df.filter(regex='-SPEC$').columns)]
displayComparison(summary_df)

## Details

This section shows the detail data of benchmark report, base run result and test run result. 

### Detail benchmark report

In [ ]:
displayComparison(benchmark_df)

In [ ]:
# Convert testrun result Path column value to link
def make_clickable(path):
    # target _blank to open new window
    return '<a target="_blank" href="../../{}">link</a>'.format(path)

def formatRunResult(df):
    # Convert Path to link and round data
    display(df.style.format({'Path': make_clickable}).format(FORMATER, subset=['IOPS', 'LAT(ms)', 'CLAT(ms)']))

### Base run result

In [ ]:
base_df = pd.read_csv(BASE_TESTRUN_RESULT, index_col=0)
formatRunResult(base_df)

### Test run result

In [ ]:
test_df = pd.read_csv(TEST_TESTRUN_RESULT)
formatRunResult(test_df)